In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.init as init
from CycleGAN.Discriminator import Discriminator 
from CycleGAN.Generator import Discriminator 
from CycleGAN.Data import GrumpyCatDataset

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
G_x = Generator().to(device)
G_y = Generator().to(device)

D_x = Discriminator().to(device)
D_y = Discriminator().to(device)

dG_x = optim.Adam(G_x.parameters(), lr = 0.0002, weight_decay = 0.001)
dG_x.zero_grad()

dG_y = optim.Adam(G_y.parameters(), lr = 0.0002, weight_decay = 0.001)
dG_y.zero_grad()

dD_x = optim.Adam(D_x.parameters(), lr = 0.0002, weight_decay = 0.001)
dD_x.zero_grad()

dD_y = optim.Adam(D_y.parameters(), lr = 0.0002, weight_decay = 0.001)
dD_y.zero_grad()

In [55]:
dataloader = DataLoader(GrumpyCatDataset("./grumpifycat/trainA", "./grumpifycat/trainB"), batch_size=2, shuffle=True)

In [56]:
import torch
import torchvision.utils as vutils

def save_output_images(output, epoch, output_dir, batch_size):
    output_grid = vutils.make_grid(output, normalize=True, scale_each=True, nrow=batch_size//2)
    
    filename = f"output_epoch_{epoch}.png"
    file_path = os.path.join(output_dir, filename)
    vutils.save_image(output_grid, file_path)

In [57]:
gx_losses = []
dx_losses = []

gy_losses = []
dy_losses = []

G_x = G_x.to(device)
G_y = G_y.to(device)
D_x = D_x.to(device)
D_y = D_y.to(device)

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

for epoch in range(200):
    total_x_loss = 0
    total_y_loss = 0

    for i, data in tqdm(enumerate(dataloader), total=len(dataloader)):
        
        x_orig, y_orig = data
        
        x = x_orig.to(device)
        y = y_orig.to(device)

        dD_x.zero_grad()
        dD_y.zero_grad()
        dG_x.zero_grad()
        dG_y.zero_grad()

        y_gen = G_x(x).detach()
        probs_y_fake = D_y(y_gen)
        probs_y_real = D_y(y)
        dy_loss = mse_loss(probs_y_fake, torch.zeros_like(probs_y_fake).to(device)) + mse_loss(probs_y_real, torch.ones_like(probs_y_real).to(device))

        dy_loss.backward()
        dD_y.step()
        dD_y.zero_grad()
        dG_x.zero_grad()
        
        x_gen = G_y(y).detach()
        probs_x_fake = D_x(x_gen)
        probs_x_real = D_x(x)
        dx_loss = mse_loss(probs_x_fake, torch.zeros_like(probs_x_fake).to(device)) + mse_loss(probs_x_real, torch.ones_like(probs_x_real).to(device))

        dx_loss.backward()
        dD_x.step()
  

        y_gen = G_x(x)
        x_hat = G_y(y_gen)
        probs_y_fake_detached = D_y(y_gen)
        cycle_x_loss = l1_loss(x_hat, x) * 10
        gen_y_loss = mse_loss(probs_y_fake_detached, torch.ones_like(probs_y_fake_detached).to(device)) + cycle_x_loss + l1_loss(x, y_gen) * 5

        gen_y_loss.backward()
        dG_y.step()

        x_gen = G_y(y)
        y_hat = G_x(x_gen)
        probs_x_fake_detached = D_x(x_gen)
        cycle_y_loss = l1_loss(y_hat, y) * 10
        gen_x_loss = mse_loss(probs_x_fake_detached, torch.ones_like(probs_x_fake_detached).to(device)) + cycle_y_loss + l1_loss(y, x_gen) * 5

        gen_x_loss.backward()
        dG_x.step()
        
        gx = gen_x_loss.item()
        gy = gen_y_loss.item()
        dx = dx_loss.item()
        dy = dy_loss.item()

        if index == 0:
            with torch.no_grad():
                y_gen = G_x(x)
                save_output_images(y_gen, epoch, "./samples_cycles/synthesized", 4)
                save_output_images(x, epoch, "./samples_cycles/original", 4)
                save_output_images(y, epoch, "./samples_cycles/target", 4)
        
        
    gx_losses.append(gx)
    dx_losses.append(dx)
    dy_losses.append(dy)
    gy_losses.append(gy)
    
   

100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.43it/s]


100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.48it/s]


100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.45it/s]


In [59]:
torch.save(G_x.state_dict(), "pix2pix_generator_x.pt")
torch.save(G_y.state_dict(), "pix2pix_discriminator_x.pt")
torch.save(D_x.state_dict(), "pix2pix_generator_y.pt")
torch.save(D_y.state_dict(), "pix2pix_discriminator_y.pt")